# This is a demo to illustrate the general pipeline of machine learning modelling, using a real-world binary classification problem (breast cancer prediction) as an example. 

# Dataset (https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))

## The Breast Cancer dataset (WDBC) is available in machine learning repository maintained by the University of California, Irvine. The dataset contains 569 samples of malignant and benign tumor cells. The first two columns in the dataset store the unique ID numbers of the samples and the corresponding diagnosis (M=malignant, B=benign), respectively. The columns 3-32 contain 30 real-value features that have been computed from digitized images of the cell nuclei, which can be used to build a model to predict whether a tumor is benign or malignant.


# Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


# Load WDBC data and visualisation

In [ ]:
# load csv file and display some rows
all_df=pd.read_csv('WDBC.csv', index_col=False)
all_df.head()     

In [ ]:
# ID column is not useful, drop it
all_df.drop('ID', axis=1, inplace=True)
all_df.head()

In [ ]:
# use info method to have a description of the data, e.g. instances, number of features, data types
all_df.info()  

In [ ]:
# basic statistics of each column
all_df.describe()

In [ ]:
# Check the distributions of benign and malignent as labeled in Column "diagnosis"

all_df['Diagnosis'].value_counts()

In [ ]:
# Draw a bar chart for each label

sns.countplot(x="Diagnosis", data=all_df)

In [ ]:
# Use box plot to check the value range and outliers of each feature

data_mean = all_df.iloc[:, :]
data_mean.plot(kind='box', subplots=True, layout=(8,4), sharex=False, sharey=False, fontsize=12, figsize=(15,20));

In [ ]:
# Compare the features data ranges
# Only for the first 10 features, but try yourself to visualise more features

fig,ax=plt.subplots(1,figsize=(20,8))
sns.boxplot(data=all_df.iloc[:, 1:11],ax=ax)  

In [ ]:
# Use boxplots to see if certain feature can discriminate between beign and malignant

fig, axes = plt.subplots(nrows=8, ncols=4, figsize=(15,20))
fig.subplots_adjust(hspace =.2, wspace=.5)
axes = axes.ravel()
for i, col in enumerate(all_df.columns[1:]):
    _= sns.boxplot(y=col, x='Diagnosis', data=all_df, ax=axes[i])

In [ ]:
# Compute the correlation matrix to observe the correlations between pair of features.

corrMatt = all_df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corrMatt)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 12))
plt.title('Breast Cancer Feature Correlation')

# Generate a custom diverging colormap
cmap = sns.diverging_palette(260, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corrMatt, vmax=1.2, square=False, cmap=cmap, mask=mask, ax=ax, annot=True, fmt='.2g', linewidths=1);


### What can you observe from the above correlation table?
#### - The area of the tissue nucleus has a strong positive correlation with values of radius and perimeter.
#### - Some paramters are moderately positive correlated (r between 0.5-0.75) are concavity and area, concavity and perimeter etc.


In [ ]:
# Scatter plots of the first 10 "mean" features. 
# You may try to plot the other features. 

sns.pairplot(all_df[list(all_df.columns[1:11]) + ['Diagnosis']], hue="Diagnosis");

### What do you observe from the above scatter plots?
#### - Mean values of cell radius, perimeter, area, compactness, concavity and concave points can be used in classification of the cancer. Larger values of these parameters tends to show a correlation with malignant tumors.
#### - Histograms show that mean values of texture, smoothness, symmetry or fractual dimension does not show a particular preference of one diagnosis over the other.

# Data pre-processing and analysis

In [ ]:
# Assign features to X 
X = all_df.drop('Diagnosis', axis=1)

# Normalise the features to use zero mean normalisation
# only for the first 10 features, but try yourself to visualise more features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
fig,ax=plt.subplots(1,figsize=(20,8))
sns.boxplot(data=Xs,ax=ax)  

In [ ]:
# Apply PCA for dimensionality reduction
from sklearn.decomposition import PCA

feature_names = list(X.columns)
pca = PCA(n_components=10)
Xs_pca = pca.fit_transform(Xs)

In [ ]:
# Only retain the first two modes of PCA as the new features
PCA_df = pd.DataFrame()
PCA_df['PCA_1'] = Xs_pca[:,0]
PCA_df['PCA_2'] = Xs_pca[:,1]

In [ ]:
# visualise the Malignant and Benign using the two PCA features. 
plt.figure(figsize=(6,6))
plt.plot(PCA_df['PCA_1'][all_df['Diagnosis'] == 'M'],PCA_df['PCA_2'][all_df['Diagnosis'] == 'M'],'ro', alpha = 0.7, markeredgecolor = 'k')
plt.plot(PCA_df['PCA_1'][all_df['Diagnosis'] == 'B'],PCA_df['PCA_2'][all_df['Diagnosis'] == 'B'],'bo', alpha = 0.7, markeredgecolor = 'k')

plt.xlabel('PCA_1')
plt.ylabel('PCA_2')
plt.legend(['Malignant','Benign']);

# Predictive model using Support Vector Machine (SVM)

In [ ]:
# First, transform the class labels from their original string representation (M and B) into integers 1: M; 0: B

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_df['Diagnosis'] = le.fit_transform(all_df['Diagnosis'])
all_df.head()

# assign numerical label to y
y = all_df['Diagnosis']

In [ ]:
# Then stratified sampling. Divide data into training and testing sets.
# Pay attention that we are using the normalised data value Xs rather than X. You may try X.

Xs_train, Xs_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=1, stratify=y)

In [ ]:
# Use kernal SVM classifier to train a model based on 70% of the data

from sklearn.svm import SVC

clf = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', probability=True)
clf.fit(Xs_train, y_train)

In [ ]:
# Classify the test dataset and output the accuracy

classifier_score = clf.score(Xs_test, y_test)
print('The classifier accuracy score is {:03.2f}'.format(classifier_score))

In [ ]:
# Now let's try K-fold cross validation
# Get average of 5-fold cross-validation score using an SVM classifier.
# Please try different number of folds and oberve the results

from sklearn.model_selection import cross_val_score
n_folds = 5
clf_cv = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
cv_error = np.average(cross_val_score(clf_cv, Xs, y, cv=n_folds))
print('The {}-fold cross-validation accuracy score for this classifier is {:.2f}'.format(n_folds, cv_error))

In [ ]:
# Now Let's try classification with some selected features, not all the features
# With 3 features, the classification accuracy is already quite good ~95%.
# Try to include more features and observe.

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

# model with just 3 best features selected (k=3)

clf_fs_cv = Pipeline([
    ('feature_selector', SelectKBest(f_classif, k=3)),
    ('svc', SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', probability=True))
])

scores = cross_val_score(clf_fs_cv, Xs, y, cv=5)  # 5 folds.

print(scores)
avg = (100 * np.mean(scores), 100 * np.std(scores)/np.sqrt(scores.shape[0]))
print("Average score and standard deviation: (%.2f +- %.3f)%%"  %avg)


# Evaluation results

In [ ]:
# We use confusion matrix (TP, TN, FP, FN) to visualise the performance

from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.fit(Xs_train, y_train).predict(Xs_test)
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Plot confusion matrix, 
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(cm, cmap=plt.cm.Blues, alpha=0.3)
for i in range(cm.shape[0]):
     for j in range(cm.shape[1]):
         ax.text(x=j, y=i,
                s=cm[i, j], 
                va='center', ha='center')
classes=["Benign","Malignant"]
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Values', )
plt.ylabel('Actual Values');
print(classification_report(y_test, y_pred ))

In [ ]:
# Plot the receiver operating characteristic curve (ROC).
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(10,8))
probas_ = clf.predict_proba(Xs_test)
fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % (roc_auc))
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate | 1 - specificity (1 - Benign recall)')
plt.ylabel('True Positive Rate | Sensitivity (Malignant recall)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.axes().set_aspect(1);

# Now let's try some other modelling methods: Logistic regression; K nearest neighbour, Gaussian Naive Bayes, Dicision Trees, Linear Discriminant Analysis

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tqdm import tqdm_notebook as tqdm

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Test options and evaluation metric
num_folds = 5
num_instances = len(Xs_train)
scoring = 'accuracy'
results = []
names = []
for name, model in tqdm(models):
    kf = KFold(n_splits=num_folds)
    cv_results = cross_val_score(model, Xs, y, cv=kf, scoring=scoring, n_jobs=-1)
    results.append(cv_results)
    names.append(name)

print('5-Fold cross-validation accuracy score for the training data for all the classifiers') 
for name, cv_results in zip(names, results):
    print("%-10s: %.6f (%.6f)" % (name, cv_results.mean(), cv_results.std()))



In [ ]:
# Compare the algorithms
plt.title( 'Algorithm Comparison' )
plt.boxplot(results)
plt.xlabel('Classifiers')
plt.ylabel('5-Fold CV Scores')
plt.xticks(np.arange(len(names)) + 1, names);

## It is observed that for this application, a simple linear regression works better than non-linear models
## Here're some practice for you to try
## 1- Observe the results if we don't normalise the feature value range
## 2- Currently all the modelling methods (i.e. SVM, KNN, LG, etc) used default parameter settings. Check the related documents and try different settings to see if the performance can be improved.
## 3- Check other feature selection methods and compare if the automatically selected features are consistent with the obervations using the correlation plots and box plots. 